In [29]:
import os,sys
import pandas as pd
from sklearn.model_selection import train_test_split
import scipy.io
import numpy as np 
from scipy.spatial.distance import pdist
import time 
import math
from sklearn.model_selection import GridSearchCV
import keras
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn import ensemble
from sklearn.metrics import accuracy_score, make_scorer
from statistics import mean
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report



## Provide directories for training/testing images.

In [23]:
root = sys.path[0]
train_dir =  os.path.join(root,  '../data/train_set')  
train_image_dir =  os.path.join(train_dir, 'images')
train_pt_dir =  os.path.join(train_dir, 'points' )
train_label_path =  os.path.join(train_dir,  "label.csv")

In [38]:
info = pd.read_csv(train_label_path)
# read mat file and store coordinates in mat 
m = []
for idx in info['Index']: 
    file = "%04d.mat"%(idx)
    m.append( scipy.io.loadmat( os.path.join( train_pt_dir, file ) ))

mat = [x[[i for i in x.keys() if not i in ['__header__', '__version__', '__globals__']][0]] for x in m]
c = np.array([pdist(x) for x in mat[0:]])

##  Train/Test Split Feature Extraction

In [39]:
train_idx, test_idx = train_test_split(info['Index'], test_size=0.2, random_state=123)

In [50]:
start_time_test=time.time()
train_features=np.array([pdist(mat[i-1]) for i in train_idx ])
print("baseline train features extracting takes %s seconds" % round((time.time() - start_time_test),3))
start_time_train=time.time()
test_features=np.array(([pdist(mat[i-1]) for i in test_idx ]))
print("base line test features extracting takes %s seconds" % round((time.time() - start_time_train),3))
train_labels=info.emotion_idx[train_idx-1]
test_labels=info.emotion_idx[test_idx-1]

print(train_features1.shape,train_labels.shape)

baseline train features extracting takes 0.11 seconds
base line test features extracting takes 0.028 seconds
(2000, 3003) (2000,)


array([[ 41.28131013,  24.68006993,   9.18497095, ...,  62.86678538,
        123.26141733,  60.46156566],
       [ 36.43413354,  22.73971842,  20.00542935, ...,  54.12569702,
        110.2829837 ,  56.27797075],
       [ 32.57281553,  25.77528199,  15.76825764, ...,  46.168278  ,
         89.68972363,  43.69834798],
       ...,
       [ 34.33209563,  25.69399343,  17.99209571, ...,  53.89414218,
        131.5742698 ,  77.68056242],
       [ 33.91603194,  19.98838908,   9.72940058, ...,  59.42931385,
        110.1260023 ,  50.90291262],
       [ 34.96800445,  25.13273624,   8.11724892, ...,  40.18621579,
         90.96279214,  50.94776961]])

##  Train a Bseline GBM model with training features and responses

In [48]:
#baseline GBM
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


baseline = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100,max_depth=3, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10)
start_time=time.time()
baseline.fit(train_features, train_labels)
print("training  model takes %s seconds" % round((time.time() - start_time),3))
predictors=list(train_features)


print('Accuracy of the GBM on test set: {:.3f}'.format(baseline.score(test_features,test_labels)))
start_time1 = time.time()
pred=baseline.predict(test_features)
print("testing model takes %s seconds" % round((time.time() - start_time1),3))
print(classification_report(test_labels, pred))

training  model takes 26.562 seconds
Accuracy of the GBM on test set: 0.440
testing model takes 0.072 seconds
              precision    recall  f1-score   support

           1       0.50      0.56      0.53        18
           2       0.65      0.68      0.67        19
           3       0.40      0.56      0.47        25
           4       0.50      0.52      0.51        21
           5       0.58      0.61      0.59        18
           6       0.63      0.46      0.53        26
           7       0.40      0.50      0.44        20
           8       0.73      0.69      0.71        16
           9       0.78      0.56      0.65        25
          10       0.45      0.45      0.45        20
          11       0.41      0.50      0.45        24
          12       0.44      0.25      0.32        32
          13       0.12      0.17      0.14        24
          14       0.54      0.57      0.55        23
          15       0.59      0.45      0.51        22
          16       0.73  

# Parameter tuning

### parameter tuning Learning_rate,n_estimator

In [77]:
#parameter tuning Learning_rate,n_estimator
#p_test3 = {'learning_rate':[0.1,0.05], 'n_estimators':[50,100,250,500,750,1000,1250,1500,1750]}

#tuning = GridSearchCV(estimator =GradientBoostingClassifier(max_depth=4, min_samples_split=2, min_samples_leaf=1, subsample=1,max_features='sqrt', random_state=10), 
                param_grid = p_test3, scoring='accuracy',n_jobs=4,iid=False, cv=5)
#tuning.fit(train_features,train_labels)
#tuning.cv_results_, tuning.best_params_, tuning.best_score_



({'mean_fit_time': array([ 15.40715823,  29.28902969,  49.3944324 ,  53.11507711,
          55.18816986,  59.12221122,  61.25709095,  64.76929288,
          67.08024154,  14.92883058,  29.83723369,  71.5798696 ,
          99.23926091, 102.55658336, 105.28049517, 107.4824029 ,
         111.50146308, 111.21266351]),
  'std_fit_time': array([0.17391099, 0.19572229, 0.69058842, 1.09710315, 0.39118344,
         0.51806917, 0.888164  , 0.8851804 , 0.87036257, 0.16340271,
         0.21745172, 0.2978652 , 1.01106934, 1.54087285, 0.79941837,
         0.99481422, 2.61529059, 4.92342539]),
  'mean_score_time': array([0.01815162, 0.03510671, 0.04468083, 0.0594408 , 0.06123633,
         0.06302385, 0.06722059, 0.07380252, 0.09155502, 0.01875072,
         0.03091779, 0.06203451, 0.09354968, 0.09195414, 0.09833708,
         0.10930796, 0.12526441, 0.1120997 ]),
  'std_score_time': array([0.00159578, 0.00801353, 0.0023083 , 0.0121594 , 0.00559242,
         0.00394756, 0.0050699 , 0.00581488, 0.0231197

### Tuning Max_depth, Min_samples_split

In [15]:

#param_test2 = {'max_depth':range(1,16,2), 'min_samples_split':range(2,102,20)}
#tuning2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_features='sqrt', subsample=1, random_state=10), 
#param_grid = param_test2, scoring='accuracy',n_jobs=4,iid=False, cv=5)
#tuning2.fit(train_features,train_labels)
#tuning2.cv_results_, tuning2.best_params_, tuning2.best_score_


({'mean_fit_time': array([ 40.58575516,  38.35696554,  38.57977009,  38.29952092,
          38.31687417, 102.51422338, 102.32999001, 101.57649493,
         101.21041594, 102.14507036,  89.53472524, 102.88205466,
         144.803303  , 160.99206958, 167.96524396, 112.35107007,
         122.19003735, 140.52957344, 150.18778157, 155.27474465,
         113.75140457, 126.88974538, 142.95140014, 147.15658612,
         150.66461711, 115.68851538, 121.36540375, 138.62919407,
         151.98337736, 157.94876437, 116.13507071, 125.74460964,
         139.67533135, 151.67527475, 133.13618879,  75.14852123,
          92.01279664,  99.80096526, 107.34059048, 107.58712349]),
  'std_fit_time': array([ 0.92650797,  0.20145001,  0.12937892,  0.14956558,  0.15791415,
          0.91100359,  0.19957817,  0.22300143,  0.32114387,  1.9639527 ,
          1.48237517,  9.51006408,  7.70020316,  2.31695772,  1.83626638,
          0.86618724,  3.69189735,  2.17396272,  1.2243321 ,  1.84006736,
          1.4079645

### Parameter Tuning:min_samples_split,min_samples_leaf

In [16]:
#param_test3 = {'min_samples_split':range(2,102,20), 'min_samples_leaf':range(30,71,10)}
#tuning3 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500,max_depth=5,min_samples_split=62,max_features='sqrt', subsample=1, random_state=10), 
#param_grid = param_test3,scoring='accuracy',n_jobs=4,iid=False, cv=5)
#tuning3.fit(train_features, train_labels)
#tuning3.cv_results_, tuning3.best_params_, tuning3.best_score_

({'mean_fit_time': array([ 89.67760768,  83.53989534,  82.02235537,  84.85673394,
          91.12463117,  88.45990372,  88.95581708,  88.4936264 ,
          87.9564497 ,  89.01561599,  94.26716447,  95.08590298,
         105.26442308,  93.48964472,  93.88019991,  96.77465463,
          97.0880167 ,  97.57455482,  97.78638864,  97.30277658,
         100.91138659, 100.74981833, 100.65866098, 100.09497709,
          95.46556411]),
  'std_fit_time': array([1.49615454, 0.95966231, 0.88159121, 1.46604421, 0.6569406 ,
         1.16994838, 1.08912517, 1.1969786 , 0.94302349, 1.09887796,
         1.0402497 , 2.65640562, 5.74684138, 1.47652983, 1.14564666,
         0.45215431, 0.9244861 , 1.88849509, 1.64256725, 1.35882913,
         1.26310973, 1.02911607, 1.10146331, 1.15709989, 8.16125676]),
  'mean_score_time': array([0.088762  , 0.10112958, 0.09614301, 0.09015217, 0.1075119 ,
         0.10511837, 0.11090388, 0.11329641, 0.09195518, 0.10711389,
         0.10192838, 0.13703365, 0.10192704, 0.1

### Parameter Tuning Max_features

In [17]:
#param_test4 = {'max_features':range(7,20,2)}
#tuning4 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500,max_depth=5, min_samples_split=62, min_samples_leaf=30, subsample=0.8, random_state=10),
#param_grid = param_test4, scoring='accuracy',n_jobs=4,iid=False, cv=5)
#tuning4.fit(train_features,train_labels)
#tuning4.cv_results_, tuning4.best_params_, tuning4.best_score_

({'mean_fit_time': array([26.61369672, 27.64383807, 29.97579255, 32.50901365, 34.85553598,
         37.64068222, 37.93309979]),
  'std_fit_time': array([0.62323588, 0.27636795, 0.2664762 , 0.32661503, 0.21860567,
         0.41865095, 1.87894825]),
  'mean_score_time': array([0.17892132, 0.16435418, 0.13005219, 0.13464012, 0.11389446,
         0.11728668, 0.10691428]),
  'std_score_time': array([0.01577802, 0.03832768, 0.01047066, 0.01266297, 0.00452154,
         0.00768974, 0.01401119]),
  'param_max_features': masked_array(data=[7, 9, 11, 13, 15, 17, 19],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_features': 7},
   {'max_features': 9},
   {'max_features': 11},
   {'max_features': 13},
   {'max_features': 15},
   {'max_features': 17},
   {'max_features': 19}],
  'split0_test_score': array([0.4464752 , 0.45953003, 0.45691906, 0.46997389, 0.44386423,
         0.45430809, 0.45169713]),
  '

### Parameter Tuning Subsample

In [18]:
#param_test5 = {'subsample':[0.6,0.7,0.75,0.8,0.85,0.9,1.0]}
#tuning5 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500,max_depth=5,min_samples_split=62, min_samples_leaf=30,random_state=10,max_features='sqrt'),
#param_grid = param_test5, scoring='accuracy',n_jobs=4,iid=False, cv=5)
#tuning5.fit(train_features,train_labels)
#tuning5.cv_results_, tuning5.best_params_, tuning5.best_score_

({'mean_fit_time': array([84.12480321, 82.41277676, 82.12166862, 81.9759521 , 82.18908262,
         83.05994167, 79.0576488 ]),
  'std_fit_time': array([0.1310422 , 0.52492128, 0.7460151 , 1.16720529, 0.88341121,
         0.57489907, 4.32177526]),
  'mean_score_time': array([0.17453294, 0.13842959, 0.11428852, 0.12825618, 0.11329751,
         0.09574437, 0.0987359 ]),
  'std_score_time': array([0.05270554, 0.02701596, 0.01430072, 0.03592944, 0.0267132 ,
         0.00459232, 0.02566052]),
  'param_subsample': masked_array(data=[0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 1.0],
               mask=[False, False, False, False, False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'subsample': 0.6},
   {'subsample': 0.7},
   {'subsample': 0.75},
   {'subsample': 0.8},
   {'subsample': 0.85},
   {'subsample': 0.9},
   {'subsample': 1.0}],
  'split0_test_score': array([0.45953003, 0.45169713, 0.50130548, 0.46736292, 0.47519582,
         0.46214099, 0.48302872]),
  'sp

### Final Parameter set at: learning_rate=0.05, n_estimators=500,max_depth=5,min_samples_split=62, min_samples_leaf=30,random_state=10,max_features='sqrt',subsample=1.0

In [51]:




baseline_tune=GradientBoostingClassifier(learning_rate=0.05, n_estimators=500,max_depth=5,min_samples_split=62, min_samples_leaf=30,random_state=10,max_features='sqrt',subsample=1.0)
start_time=time.time()
baseline_tune.fit(train_features, train_labels)
print("training  model takes %s seconds" % round((time.time() - start_time),3))
predictors=list(train_features)


print('Accuracy of the GBM on test set: {:.3f}'.format(baseline_tune.score(test_features,test_labels)))
start_time1 = time.time()
pred=baseline_tune.predict(test_features)
print("testing model takes %s seconds" % round((time.time() - start_time1),3))
print(classification_report(test_labels, pred))

training  model takes 105.502 seconds
Accuracy of the GBM on test set: 0.486
testing model takes 0.103 seconds
              precision    recall  f1-score   support

           1       0.57      0.72      0.63        18
           2       0.70      0.84      0.76        19
           3       0.41      0.52      0.46        25
           4       0.43      0.57      0.49        21
           5       0.57      0.72      0.63        18
           6       0.72      0.50      0.59        26
           7       0.58      0.55      0.56        20
           8       0.75      0.75      0.75        16
           9       0.80      0.64      0.71        25
          10       0.41      0.45      0.43        20
          11       0.48      0.67      0.56        24
          12       0.44      0.34      0.39        32
          13       0.22      0.21      0.21        24
          14       0.52      0.65      0.58        23
          15       0.61      0.50      0.55        22
          16       0.80 

### Increase accuracy from 0.440 to 0.486 after tuning